Last updated: 16 Feb 2023

# 👋 PyCaret Time Series Forecasting Tutorial

PyCaret is an open-source, low-code machine learning library in Python that automates machine learning workflows. It is an end-to-end machine learning and model management tool that exponentially speeds up the experiment cycle and makes you more productive.

Compared with the other open-source machine learning libraries, PyCaret is an alternate low-code library that can be used to replace hundreds of lines of code with a few lines only. This makes experiments exponentially fast and efficient. PyCaret is essentially a Python wrapper around several machine learning libraries and frameworks, such as scikit-learn, XGBoost, LightGBM, CatBoost, spaCy, Optuna, Hyperopt, Ray, and a few more.

The design and simplicity of PyCaret are inspired by the emerging role of citizen data scientists, a term first used by Gartner. Citizen Data Scientists are power users who can perform both simple and moderately sophisticated analytical tasks that would previously have required more technical expertise.


# 💻 Installation

PyCaret is tested and supported on the following 64-bit systems:
- Python 3.7 – 3.10
- Python 3.9 for Ubuntu only
- Ubuntu 16.04 or later
- Windows 7 or later

You can install PyCaret with Python's pip package manager:

`pip install pycaret`

PyCaret's default installation will not install all the extra dependencies automatically. For that you will have to install the full version:

`pip install pycaret[full]`

or depending on your use-case you may install one of the following variant:

- `pip install pycaret[analysis]`
- `pip install pycaret[models]`
- `pip install pycaret[tuner]`
- `pip install pycaret[mlops]`
- `pip install pycaret[parallel]`
- `pip install pycaret[test]`

In [ ]:
# install pycaret
#!pip install pycaret[full]
# check installed version
import pycaret 
pycaret.__version__

# 🚀 Quick start

PyCaret's time series forecasting module is now available. The module currently is suitable for univariate / multivariate time series forecasting tasks. The API of time series module is consistent with other modules of PyCaret.

It comes built-in with preprocessing capabilities and over 30 algorithms comprising of statistical / time-series methods as well as machine learning based models. In addition to the model training, this module has lot of other capabilities such as automated hyperparameter tuning, ensembling, model analysis, model packaging and deployment capabilities. 

A typical workflow in PyCaret consist of following 5 steps in this order:

### **Setup** ➡️ **Compare Models** ➡️ **Analyze Model** ➡️ **Prediction** ➡️ **Save Model** <br/>

In [ ]:
# import pandas module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
# creating a data frame
data = pd.read_csv("binned_removed.csv",header=0)
data.head()

# mean of one column
data_mean = data["638de53168f31919048f189d, 63932c6e68f319085df375b5; B2_solar_x2_03, Soil_tension"].mean()
print(data_mean)

In [ ]:
# plot the dataset
data.plot(figsize = (20,10))
fig = plt.gcf()
fig.savefig('all_data.png')

In [ ]:
# Show if there are any missing values inside the data
data.isna().any()

In [ ]:
import missingno as msno 
# Plot correlation heatmap of missingness
msno.matrix(data);

In [ ]:
msno.heatmap(data);

In [ ]:
msno.dendrogram(data)

In [ ]:
# Get length cols
len(data.columns)
print(data.head())

In [ ]:
from sklearn.impute import KNNImputer

# Copy the data
data = data.copy(deep=True)

#data_knn_imputed = data_knn_imputed.drop(['Time'], axis=1)

# Init the transformer
knn_imp = KNNImputer(n_neighbors=3)

# Fit/transform
data.iloc[:,1:8] = knn_imp.fit_transform(data.iloc[:,1:8])

# Plot correlation heatmap of missingness
msno.matrix(data)

In [ ]:
data.tail()

In [ ]:
# Show if there are any missing values inside the data
data.isna().any()

In [ ]:
# create average cols
data['grouped_soil'] = data[['638de53168f31919048f189d, 63932c6e68f319085df375b5; B2_solar_x2_03, Soil_tension', '638de57568f31919048f189f, 63932c3668f319085df375ab; B4_solar_x1_03, Soil_tension']].mean(axis=1)
data['grouped_resistance'] = data[['638de53168f31919048f189d, 63932c6e68f319085df375b7; B2_solar_x2_03, Resistance', '638de56a68f31919048f189e, 63932ce468f319085df375cb; B3_solar_x1_02, Resistance', '638de57568f31919048f189f, 63932c3668f319085df375ad; B4_solar_x1_03, Resistance']].mean(axis=1)
data['grouped_soil_temp'] = data[['638de53168f31919048f189d, 63932c6e68f319085df375b9; B2_solar_x2_03, Soil_temperature', '638de57568f31919048f189f, 63932c3668f319085df375af; B4_solar_x1_03, Soil_temperature']].mean(axis=1)


In [ ]:
data.dtypes

In [ ]:
#for test
data_arr = [{'mydate': '2007-02-01T05:00:00+0000 UTC', 'b': 2, 'c':3},
        {'mydate': '2023-02-08 16:10:00 +0000 UTC', 'b': 20, 'c': 30},
        {'mydate': '2007-02-01T05:00:00+0000 UTC', 'b': 20, 'c': 30}]

# Creates DataFrame.
df = pd.DataFrame(data_arr)

# Below line commented to adopt simpler alternative
# df['mydate'] = pd.to_datetime(df['mydate'], format="%Y-%m-%d %H:%M:%S.%f")
# Simplified version due to Mike T comment
df['mydate'] = df['mydate'].str[:-4]
df['mydate'] = pd.to_datetime(df['mydate'])


df.head()

In [ ]:
df.dtypes

In [ ]:
data['Time'] = data['Time'].str[:-9]
data.head()

In [ ]:
data['Time']=pd.to_datetime(data['Time'])

data.head()

In [ ]:
data.dtypes

In [ ]:
## Create Features
## create rolling mean: introduces NaN again -> later just cut off
data['rolling_mean_grouped_soil'] = data['grouped_soil'].rolling(window=10, win_type='gaussian').mean(std=10)
data['rolling_mean_grouped_soil_temp'] = data['grouped_soil_temp'].rolling(window=10, win_type='gaussian').mean(std=10)

data['hour'] = data['Time'].dt.hour
data['minute'] = data['Time'].dt.minute
data['date'] = data['Time'].dt.day
data['month'] = data['Time'].dt.month

#data['Time'] = data.index

data = data[['Time', 'hour', 'minute', 'date', 'month', 'grouped_soil', 'grouped_resistance', 'grouped_soil_temp', 'rolling_mean_grouped_soil', 'rolling_mean_grouped_soil_temp', '638de53168f31919048f189d, 63932c6e68f319085df375b5; B2_solar_x2_03, Soil_tension', '638de53168f31919048f189d, 63932c6e68f319085df375b7; B2_solar_x2_03, Resistance', '638de53168f31919048f189d, 63932c6e68f319085df375b9; B2_solar_x2_03, Soil_temperature', '638de56a68f31919048f189e, 63932ce468f319085df375cb; B3_solar_x1_02, Resistance', '638de57568f31919048f189f, 63932c3668f319085df375ab; B4_solar_x1_03, Soil_tension', '638de57568f31919048f189f, 63932c3668f319085df375ad; B4_solar_x1_03, Resistance', '638de57568f31919048f189f, 63932c3668f319085df375af; B4_solar_x1_03, Soil_temperature']]

data.head(10)

In [ ]:
data.dtypes

In [ ]:
data_plot = data.drop(['hour','minute','date','month','grouped_soil','grouped_resistance','grouped_soil_temp','rolling_mean_grouped_soil_temp','638de53168f31919048f189d, 63932c6e68f319085df375b5; B2_solar_x2_03, Soil_tension','638de53168f31919048f189d, 63932c6e68f319085df375b7; B2_solar_x2_03, Resistance','638de53168f31919048f189d, 63932c6e68f319085df375b9; B2_solar_x2_03, Soil_temperature','638de56a68f31919048f189e, 63932ce468f319085df375cb; B3_solar_x1_02, Resistance','638de57568f31919048f189f, 63932c3668f319085df375ab; B4_solar_x1_03, Soil_tension','638de57568f31919048f189f, 63932c3668f319085df375ad; B4_solar_x1_03, Resistance','638de57568f31919048f189f, 63932c3668f319085df375af; B4_solar_x1_03, Soil_temperature'], axis=1)
data_plot[10:].head(20)

In [ ]:
data_plot.set_index('Time').plot(figsize = (20,10))
fig = plt.gcf()
fig.savefig('rolling_mean_grouped_soil.png')

In [ ]:
# Split dataset into train and test set
def split_data(data, split_date):
    return data[data['Time'] <= split_date].copy(), \
           data[data['Time'] >  split_date].copy()

In [ ]:
train, test = split_data(data[10:], pd.to_datetime('2023-03-08 15:30')) # splitting the data for training before 15th June

plt.figure(figsize=(20,10))
plt.xlabel('Time')
plt.ylabel('rolling_mean_grouped_soil')
plt.plot(train.index,train['rolling_mean_grouped_soil'],label='train')
plt.plot(test.index,test['rolling_mean_grouped_soil'],label='test')
plt.legend()
plt.show()

## Setup
This function initializes the training environment and creates the transformation pipeline. Setup function must be called before executing any other function in PyCaret. `Setup` has only one required parameter i.e. `data`. All the other parameters are optional.

In [ ]:
train.head()

In [ ]:
# import pycaret time series and init setup
from pycaret.time_series import *
s = setup(data[10:], target = 'rolling_mean_grouped_soil', enforce_exogenous = False, fh = 10, session_id = 123, ignore_features = ['Time'], numeric_imputation_exogenous = 'mean') # currently I let setup function do the slit of data in train and test

Once the setup has been successfully executed it shows the information grid containing experiment level information. 

- **Session id:**  A pseudo-random number distributed as a seed in all functions for later reproducibility. If no `session_id` is passed, a random number is automatically generated that is distributed to all functions.<br/>
<br/>
- **Approach:**  Univariate or multivariate. <br/>
<br/>
- **Exogenous Variables:**  Exogeneous variables to be used in model. <br/>
<br/>
- **Original data shape:**  Shape of the original data prior to any transformations. <br/>
<br/>
- **Transformed train set shape :**  Shape of transformed train set <br/>
<br/>
- **Transformed test set shape :**  Shape of transformed test set <br/>
<br/>

PyCaret has two set of API's that you can work with. (1) Functional (as seen above) and (2) Object Oriented API.

With Object Oriented API instead of executing functions directly you will import a class and execute methods of class.

In [ ]:
# import TSForecastingExperiment and init the class
from pycaret.regression import *
from pycaret.time_series import *
exp = TSForecastingExperiment()

In [ ]:
# check the type of exp
type(exp)

In [ ]:
# init setup on exp
exp.setup(data[10:], target='rolling_mean_grouped_soil', enforce_exogenous = False, fh = 10, session_id = 123, ignore_features = ['Time'], numeric_imputation_exogenous = 'mean')

In [ ]:
exp.plot_model(plot="train_test_split")

In [ ]:
# show available models
exp.models()

You can use any of the two method i.e. Functional or OOP and even switch back and forth between two set of API's. The choice of method will not impact the results and has been tested for consistency.

## Check Stats
The `check_stats` function is used to get summary statistics and run statistical tests on the original data or model residuals.

In [ ]:
# check statistical tests on original data
check_stats()

## Compare Models

This function trains and evaluates the performance of all the estimators available in the model library using cross-validation. The output of this function is a scoring grid with average cross-validated scores. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# compare baseline models
best = compare_models(n_select = 20, fold = 3, sort = 'R2', verbose = 1, exclude=['arima', 'auto_arima'])

In [ ]:
print(best)

In [ ]:
# compare models using OOP
exp.compare_models(exclude=['arima', 'auto_arima'])

Notice that the output between functional and OOP API is consistent. Rest of the functions in this notebook will only be shown using functional API only. 

___

## Analyze Model

You can use the `plot_model` function to analyzes the performance of a trained model on the test set. It may require re-training the model in certain cases.

In [ ]:
# plot forecast
plot_model(best[0], plot = 'forecast')

In [ ]:
# plot forecast for 36 months in future
plot_model(best[0], plot = 'forecast', data_kwargs = {'fh' : 709})

In [ ]:
# residuals plot
plot_model(best, plot = 'residuals')

In [ ]:
# check docstring to see available plots 
help(plot_model)

An alternate to `plot_model` function is `evaluate_model`. It can only be used in Notebook since it uses ipywidget.

## Prediction
The `predict_model` function returns `y_pred`. When data is `None` (default), it uses `fh` as defined during the `setup` function. 

In [ ]:
# predict on test set
holdout_pred = predict_model(best[1])

In [ ]:
# show predictions df
holdout_pred.head()

In [ ]:
# generate forecast for 36 period in future
predict_model(best[0], fh = 36)

## Save Model

Finally, you can save the entire pipeline on disk for later use, using pycaret's `save_model` function.

In [ ]:
# save pipeline
save_model(best[0], 'my_first_pipeline_TS')

In [ ]:
# load pipeline
loaded_best_pipeline = load_model('my_first_pipeline')
loaded_best_pipeline

# 👇 Detailed function-by-function overview

## ✅ Setup
This function initializes the training environment and creates the transformation pipeline. Setup function must be called before executing any other function in PyCaret. `Setup` has only one required parameter i.e. `data`. All the other parameters are optional.

In [ ]:
s = setup(data[10:], target='rolling_mean_grouped_soil', fh = 709, session_id = 123, use_gpu = True, ignore_features = ['Time'])

To access all the variables created by the setup function such as transformed dataset, random_state, etc. you can use `get_config` method.

In [ ]:
# check all available config
get_config()

In [ ]:
# lets access y_train_transformed
get_config('y_train_transformed')

In [ ]:
# another example: let's access seed
print("The current seed is: {}".format(get_config('seed')))

# now lets change it using set_config
set_config('seed', 786)
print("The new seed is: {}".format(get_config('seed')))

All the preprocessing configurations and experiment settings/parameters are passed into the `setup` function. To see all available parameters, check the docstring:

In [ ]:
# help(setup)

In [ ]:
# init setup fold_strategy = expanding
s = setup(train, target='rolling_mean_grouped_soil', use_gpu = True, fh = 3, session_id = 123,
          fold_strategy = 'expanding', numeric_imputation_target = 'drift', ignore_features = ['Time'])

## ✅ Compare Models
This function trains and evaluates the performance of all estimators available in the model library using cross-validation. The output of this function is a scoring grid with average cross-validated scores. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
best = compare_models(exclude=['arima', 'auto_arima'])

`compare_models` by default uses all the estimators in model library (all except models with `Turbo=False`) . To see all available models you can use the function `models()`

In [ ]:
# check available models
models()

You can use the `include` and `exclude` parameter in the `compare_models` to train only select model or exclude specific models from training by passing the model id's in `exclude` parameter.

In [ ]:
compare_ts_models = compare_models(include = ['ets', 'arima', 'theta', 'naive', 'snaive', 'grand_means', 'polytrend'])

In [ ]:
compare_ts_models

The function above has return trained model object as an output. The scoring grid is only displayed and not returned. If you need access to the scoring grid you can use `pull` function to access the dataframe.

In [ ]:
compare_ts_models_results = pull()
compare_ts_models_results

By default `compare_models` return the single best performing model based on the metric defined in the `sort` parameter. Let's change our code to return 3 top models based on `MAE`.

In [ ]:
best_mae_models_top3 = compare_models(sort = 'R2', n_select = 3, exclude=['arima', 'auto_arima'])

In [ ]:
# list of top 3 models by MAE
best_mae_models_top3

Some other parameters that you might find very useful in `compare_models` are:

- fold
- cross_validation
- budget_time
- errors
- parallel
- engine

You can check the docstring of the function for more info.

In [ ]:
# help(compare_models)

## ✅  Check Stats
The `check_stats` function is used to get summary statistics and run statistical tests on the original data or model residuals.

In [ ]:
# check stats on original data
check_stats()

In [ ]:
# check_stats on residuals of best model
check_stats(estimator = best)

## ✅ Experiment Logging
PyCaret integrates with many different type of experiment loggers (default = 'mlflow'). To turn on experiment tracking in PyCaret you can set `log_experiment` and `experiment_name` parameter. It will automatically track all the metrics, hyperparameters, and artifacts based on the defined logger.

In [ ]:
# from pycaret.time_series import *
# s = setup(data, fh = 3, session_id = 123, log_experiment='mlflow', experiment_name='airline_experiment')

In [ ]:
# compare models
# best = compare_models()

In [ ]:
# start mlflow server on localhost:5000
# !mlflow ui

By default PyCaret uses `MLFlow` logger that can be changed using `log_experiment` parameter. Following loggers are available:
    
    - mlflow
    - wandb
    - comet_ml
    - dagshub
    
Other logging related parameters that you may find useful are:

- experiment_custom_tags
- log_plots
- log_data
- log_profile

For more information check out the docstring of the `setup` function.

In [ ]:
# help(setup)

## ✅ Create Model
This function trains and evaluates the performance of a given estimator using cross-validation. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function. All the available models can be accessed using the models function.

In [ ]:
# check all the available models
models()

In [ ]:
# train ets with default fold=3
ets = create_model('br_cds_dt')

The function above has return trained model object as an output. The scoring grid is only displayed and not returned. If you need access to the scoring grid you can use `pull` function to access the dataframe.

In [ ]:
ets_results = pull()
print(type(ets_results))
ets_results

In [ ]:
# train theta model with fold=5
theta = create_model('theta', fold=5)

In [ ]:
# train theta with specific model parameters
create_model('theta', deseasonalize = False, fold=5)

Some other parameters that you might find very useful in `create_model` are:

- cross_validation
- engine
- fit_kwargs

You can check the docstring of the function for more info.

In [ ]:
# help(create_model)

## ✅ Tune Model

The `tune_model` function tunes the hyperparameters of the model. The output of this function is a scoring grid with cross-validated scores by fold. The best model is selected based on the metric defined in optimize parameter. Metrics evaluated during cross-validation can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# train a dt model with default params
dt = create_model('br_cds_dt')

In [ ]:
# tune hyperparameters of dt
tuned_dt = tune_model(dt)

Metric to optimize can be defined in `optimize` parameter (default = 'MASE'). Also, a custom tuned grid can be passed with `custom_grid` parameter. 

In [ ]:
dt

In [ ]:
# define tuning grid
dt_grid = {'regressor__max_depth' : [None, 2, 4, 6, 8, 10, 12]}

# tune model with custom grid and metric = MAE
tuned_dt = tune_model(dt, custom_grid = dt_grid, optimize = 'MAE')

In [ ]:
# see tuned_dt params
tuned_dt

In [ ]:
# to access the tuner object you can set return_tuner = True
tuned_dt, tuner = tune_model(dt, return_tuner=True)

In [ ]:
# model object
tuned_dt

In [ ]:
# tuner object
tuner

For more details on all available `search_library` and `search_algorithm` please check the docstring. Some other parameters that you might find very useful in `tune_model` are:

- choose_better
- custom_scorer
- n_iter
- search_algorithm
- optimize

You can check the docstring of the function for more info.

In [ ]:
# help(tune_model)

## ✅ Blend Models

This function trains a `EnsembleForecaster` for select models passed in the estimator_list parameter. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# top 3 models based on mae
best_mae_models_top3

In [ ]:
# blend top 3 models
blend_models(best_mae_models_top3)

Some other parameters that you might find very useful in `blend_models` are:

- choose_better
- method
- weights
- fit_kwargs
- optimize

You can check the docstring of the function for more info.

In [ ]:
# help(blend_models)

## ✅ Plot Model

This function analyzes the performance of a trained model on the hold-out set. It may require re-training the model in certain cases.

In [ ]:
# plot forecast
plot_model(best, plot = 'forecast')

In [ ]:
# plot acf
# for certain plots you don't need a trained model
plot_model(plot = 'acf')

In [ ]:
# plot diagnostics
# for certain plots you don't need a trained model
plot_model(plot = 'diagnostics')

Some other parameters that you might find very useful in `plot_model` are:

- fig_kwargs
- data_kwargs
- display_format
- return_fig
- return_data
- save

You can check the docstring of the function for more info.

In [ ]:
# help(plot_model)

## ✅ Finalize Model
This function trains a given model on the entire dataset including the hold-out set.

In [ ]:
final_best = finalize_model(best)

In [ ]:
final_best

## ✅ Deploy Model
This function deploys the entire ML pipeline on the cloud.

**AWS:**  When deploying model on AWS S3, environment variables must be configured using the command-line interface. To configure AWS environment variables, type `aws configure` in terminal. The following information is required which can be generated using the Identity and Access Management (IAM) portal of your amazon console account:

- AWS Access Key ID
- AWS Secret Key Access
- Default Region Name (can be seen under Global settings on your AWS console)
- Default output format (must be left blank)

**GCP:** To deploy a model on Google Cloud Platform ('gcp'), the project must be created using the command-line or GCP console. Once the project is created, you must create a service account and download the service account key as a JSON file to set environment variables in your local environment. Learn more about it: https://cloud.google.com/docs/authentication/production

**Azure:** To deploy a model on Microsoft Azure ('azure'), environment variables for the connection string must be set in your local environment. Go to settings of storage account on Azure portal to access the connection string required.
AZURE_STORAGE_CONNECTION_STRING (required as environment variable)
Learn more about it: https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?toc=%2Fpython%2Fazure%2FTOC.json

In [ ]:
# deploy model on aws s3
# deploy_model(best, model_name = 'my_first_platform_on_aws',
#             platform = 'aws', authentication = {'bucket' : 'pycaret-test'})

In [ ]:
# load model from aws s3
# loaded_from_aws = load_model(model_name = 'my_first_platform_on_aws', platform = 'aws',
#                              authentication = {'bucket' : 'pycaret-test'})

# loaded_from_aws

## ✅ Save / Load Model
This function saves the transformation pipeline and a trained model object into the current working directory as a pickle file for later use.

In [ ]:
# save model
save_model(best, 'my_first_model_TS')

In [ ]:
# load model
loaded_from_disk = load_model('my_first_model_TS')
loaded_from_disk

## ✅ Save / Load Experiment
This function saves all the experiment variables on disk, allowing to later resume without rerunning the setup function.

In [ ]:
# save experiment
save_experiment('my_experiment')

In [ ]:
# load experiment from disk
exp_from_disk = load_experiment('my_experiment', data=data) #add numerical imputation!!!!!!